# AI & Security Project

**regular_models.ipynb**: in this notebook we try out several image classification models on our dataset 'TinyImageNet'. We use a wide variety of hyperparameters


## Step 0: Configurations


In [1]:
import warnings

warnings.filterwarnings("ignore")

## Step 1. dataset and libraries


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import numpy as np


data_directory = r"./data/TinyImageNet/tiny-imagenet-200/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import os
import requests
import zipfile
import shutil

def download_and_prepare_tinyimagenet(data_dir="tiny-imagenet"):
	"""
	Download and extract the TinyImageNet dataset, and prepare it for PyTorch's ImageFolder.
	"""
	url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
	zip_file = "tiny-imagenet-200.zip"
	extract_dir = "tiny-imagenet-200"
	
	# Create the directory if it doesn't exist
	if not os.path.exists(data_dir):
		os.makedirs(data_dir)
	
	# Download the dataset
	print("Downloading TinyImageNet dataset...")
	response = requests.get(url, stream=True)
	with open(os.path.join(data_dir, zip_file), "wb") as f:
		shutil.copyfileobj(response.raw, f)
	print("Download complete.")
	
	# Extract the dataset
	print("Extracting dataset...")
	with zipfile.ZipFile(os.path.join(data_dir, zip_file), "r") as zip_ref:
		zip_ref.extractall(data_dir)
	print("Extraction complete.")
	
	# Organize validation dataset
	val_dir = os.path.join(data_dir, extract_dir, "val")
	val_images_dir = os.path.join(val_dir, "images")
	val_annotations = os.path.join(val_dir, "val_annotations.txt")
	
	# Parse the validation annotations
	print("Organizing validation dataset...")
	with open(val_annotations, "r") as f:
		lines = f.readlines()
	
	val_labels = {}
	for line in lines:
		parts = line.split("\t")
		val_labels[parts[0]] = parts[1]
	
	# Create subdirectories for validation classes
	for label in set(val_labels.values()):
		os.makedirs(os.path.join(val_dir, label), exist_ok=True)
	
	# Move validation images to corresponding class subdirectories
	for img, label in val_labels.items():
		src_path = os.path.join(val_images_dir, img)
		dst_path = os.path.join(val_dir, label, img)
		shutil.move(src_path, dst_path)
	
	# Remove the original images directory and annotation file
	shutil.rmtree(val_images_dir)
	os.remove(val_annotations)
	
	print("Dataset is ready at:", os.path.join(data_dir, extract_dir))
	return os.path.join(data_dir, extract_dir)


# Check if the dataset is already downloaded, by checking is /train/, /val/ and /test/ directories are present
downloaded = os.path.exists(data_directory + "train/")\
    and os.path.exists(data_directory + "test/")\
    and os.path.exists(data_directory + "val/")


if not downloaded:
	dataset_path = download_and_prepare_tinyimagenet(data_directory)
	print("TinyImageNet dataset is available at:", dataset_path)

# This took 3m30 on my machine

## Step 2: models

In [4]:
# Model definitions
model_definitions = {
	"ResNet18": lambda: models.resnet18(num_classes=200),
	"VGG16": lambda: models.vgg16(num_classes=200),
	"DenseNet121": lambda: models.densenet121(num_classes=200),
	"MobileNetV2": lambda: models.mobilenet_v2(num_classes=200),
}

# Model configurations (hyperparameters)
model_configs = {
    "Config1": {"lr": 0.01, "optimizer": "SGD", "momentum": 0.9, "weight_decay": 0.0005},
    "Config2": {"lr": 0.001, "optimizer": "Adam", "weight_decay": 0.0001},
    "Config3": {"lr": 0.0001, "optimizer": "AdamW", "weight_decay": 0.01}
}

# General hyperparameters
hyperparameters = {
	'batch_size': 64,
}

## Step 3: set-up training loop

In [5]:
# Step 1: Define TinyImageNet Dataset
def get_tinyimagenet_loaders(data_dir, batch_size, num_workers=4):
	"""
	Set up data loaders for TinyImageNet dataset.
	"""
	transform_train = transforms.Compose([
		transforms.RandomCrop(64, padding=4),
		transforms.RandomHorizontalFlip(),
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262])
	])

	transform_val = transforms.Compose([
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2302, 0.2265, 0.2262])
	])

	train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform_train)
	val_dataset = datasets.ImageFolder(root=f"{data_dir}/val", transform=transform_val)

	train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
	val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

	return train_loader, val_loader

# Step 2: Define Training and Evaluation Functions
def train_one_epoch(model, loader, criterion, optimizer, device):
	"""
	Train the model for one epoch.
	"""
	model.train()
	running_loss = 0.0
	correct = 0
	total = 0

	for inputs, targets in tqdm(loader, desc="Training"):
		inputs, targets = inputs.to(device), targets.to(device)
		optimizer.zero_grad()

		outputs = model(inputs)
		loss = criterion(outputs, targets)
		loss.backward()
		optimizer.step()

		running_loss += loss.item() * inputs.size(0)
		_, predicted = outputs.max(1)
		correct += predicted.eq(targets).sum().item()
		total += targets.size(0)

	epoch_loss = running_loss / total
	accuracy = correct / total
	return epoch_loss, accuracy

def evaluate(model, loader, criterion, device):
	"""
	Evaluate the model on validation data and compute top-1 and top-5 errors.
	"""
	model.eval()
	running_loss = 0.0
	top1_correct = 0
	total = 0
	top5_correct = 0

	with torch.no_grad():
		for inputs, targets in tqdm(loader, desc="Validation"):
			inputs, targets = inputs.to(device), targets.to(device)

			outputs = model(inputs)
			loss = criterion(outputs, targets)
			running_loss += loss.item() * inputs.size(0)

			_, top1_preds = outputs.max(1)
			top1_correct += top1_preds.eq(targets).sum().item()

			_, top5_preds = outputs.topk(5, 1, True, True)
			top5_correct += sum([targets[i] in top5_preds[i] for i in range(targets.size(0))])
			total += targets.size(0)

	epoch_loss = running_loss / total
	top1_error = 1 - (top1_correct / total)
	top5_error = 1 - (top5_correct / total)

	return epoch_loss, top1_error, top5_error

# Step 3: Define Model Training and Comparison
def compare_models(models, model_configs, hyperparameters, data_dir, num_epochs, device="cuda"):
	"""
	Train and evaluate multiple models with different configurations.
	"""
	results = []

	train_loader, val_loader = get_tinyimagenet_loaders(data_dir, batch_size=hyperparameters['batch_size'])

	for model_name, model_fn in models.items():
		for config_name, config in model_configs.items():
			print(f"Training {model_name} with {config_name} configuration...")
			model = model_fn().to(device)
			
			# Select optimizer based on config
			if config['optimizer'] == 'SGD':
				optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=config['momentum'], weight_decay=config['weight_decay'])
			elif config['optimizer'] == 'Adam':
				optimizer = optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
			elif config['optimizer'] == 'AdamW':
				optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])

			criterion = nn.CrossEntropyLoss()

			for epoch in range(num_epochs):
				print(f"Epoch {epoch + 1}/{num_epochs}")
				train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
				val_loss, top1_error, top5_error = evaluate(model, val_loader, criterion, device)

				print(f"Validation Loss: {val_loss:.4f}, Top-1 Error: {top1_error:.4f}, Top-5 Error: {top5_error:.4f}")
				results.append({
					'model': model_name,
					'config': config_name,
					'epoch': epoch + 1,
					'train_loss': train_loss,
					'train_acc': train_acc,
					'val_loss': val_loss,
					'top1_error': top1_error,
					'top5_error': top5_error
				})

	return results


## Step 4: train and evaluate

In [6]:
# Train and evaluate
results = compare_models(
	models=model_definitions,
	model_configs=model_configs,
	hyperparameters=hyperparameters,
	data_dir=data_directory,
	num_epochs=5,
	device=device
)

#* This took 132 minutes on my machine, using device = "cuda"

Training ResNet18 with Config1 configuration...
Epoch 1/5


Validation: 100%|██████████| 157/157 [00:32<00:00,  4.76it/s]


Validation Loss: 3.9184, Top-1 Error: 0.8504, Top-5 Error: 0.6279
Epoch 2/5


Validation: 100%|██████████| 157/157 [00:16<00:00,  9.52it/s]


Validation Loss: 3.4336, Top-1 Error: 0.7813, Top-5 Error: 0.5180
Epoch 3/5


...

Validation Loss: 4.3980, Top-1 Error: 0.9192, Top-5 Error: 0.7542
Epoch 4/5


Validation: 100%|██████████| 157/157 [00:17<00:00,  9.07it/s]


Validation Loss: 4.1813, Top-1 Error: 0.8879, Top-5 Error: 0.7014
Epoch 5/5


Validation: 100%|██████████| 157/157 [00:17<00:00,  8.97it/s]

Validation Loss: 4.0177, Top-1 Error: 0.8680, Top-5 Error: 0.6602


In [9]:
import time
import pandas as pd

# Export results to CSV
current_time = time.strftime("%Y%m%d-%H%M")
results_df = pd.DataFrame(results)
results_df.to_csv(f"./exports/data/regular_models_results_{current_time}.csv", index=False)